In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam, PosDefMatrixParam
from VariationalBayes.NormalParams import MVNParam, UVNParam, UVNParamVector
from VariationalBayes.GammaParams import GammaParam
from VariationalBayes.ExponentialFamilies import \
    UnivariateNormalEntropy, MultivariateNormalEntropy, GammaEntropy, \
    MVNPrior, UVNPrior, GammaPrior


from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy as asp
import scipy as sp

import copy
from scipy import optimize

In [2]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.
import os
import json

simulate_data = False
prior_par = ModelParamsDict('Prior Parameters')

if not simulate_data:
    #analysis_name = 'simulated_data_small'
    analysis_name = 'simulated_data_large'

    data_dir = os.path.join(os.environ['GIT_REPO_LOC'], 'LRVBLogitGLMM/LogitGLMMLRVB/inst/data/')
    json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)
    json_output_filename = os.path.join(data_dir, '%s_python_vb_results.json' % analysis_name)

    json_file = open(json_filename, 'r')
    json_dat = json.load(json_file)
    json_file.close()

    stan_dat = json_dat['stan_dat']
    vp_base = json_dat['vp_base']

    print stan_dat.keys()
    K = stan_dat['K'][0]
    NObs = stan_dat['N'][0]
    NG = stan_dat['NG'][0]
    N = NObs / NG
    y_g_vec = np.array(stan_dat['y_group'])
    y_vec = np.array(stan_dat['y'])
    x_mat = np.array(stan_dat['x'])
    
    # Define a class to contain prior parameters.
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.array(stan_dat['beta_prior_mean'])))
    beta_prior_info = np.linalg.inv(np.array(stan_dat['beta_prior_var']))
    prior_par.push_param(PosDefMatrixParam('beta_prior_info', K, val=beta_prior_info))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=stan_dat['mu_prior_mean'][0]))
    prior_par.push_param(ScalarParam('mu_prior_info', val=1 / stan_dat['mu_prior_var'][0]))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=stan_dat['tau_prior_alpha'][0]))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=stan_dat['tau_prior_beta'][0]))

    # An index set to make sure jacobians match the order expected by R.
    prior_par_indices = copy.deepcopy(prior_par)
    prior_par_indices.set_name('Prior Indices')
    prior_par_indices.set_vector(np.array(range(prior_par_indices.vector_size())))
else:
    # Simulate data instead of loading it if you like
    N = 200     # observations per group
    K = 5      # dimension of regressors
    NG = 200      # number of groups

    # Generate data
    def Logistic(u):
        return np.exp(u) / (1 + np.exp(u))

    NObs = NG * N
    true_beta = np.array(range(5))
    true_beta = true_beta - np.mean(true_beta)
    true_mu = 0.
    true_tau = 40.0
    true_u = np.random.normal(true_mu, 1 / np.sqrt(true_tau), NG)

    x_mat = np.random.random(K * NObs).reshape(NObs, K) - 0.5
    y_g_vec = [ g for g in range(NG) for n in range(N) ]
    true_rho = Logistic(np.matmul(x_mat, true_beta) + true_u[y_g_vec])
    y_vec = np.random.random(NObs) < true_rho
    
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.zeros(K)))
    prior_par.push_param(PosDefMatrixParam('beta_prior_info', K, val=0.01 * np.eye(K)))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=0))
    prior_par.push_param(ScalarParam('mu_prior_info', val=0.5))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=3.0))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=10.0))

print N * NG
print np.mean(y_vec)

[u'y_group', u'mu_prior_var', u'mu_prior_t', u'mu_prior_var_c', u'K', u'beta_prior_var', u'tau_prior_beta', u'N', u'mu_prior_mean_c', u'mu_prior_epsilon', u'mu_prior_mean', u'y', u'x', u'NG', u'beta_prior_mean', u'tau_prior_alpha']
10000
0.4668


In [3]:
# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
glmm_par = ModelParamsDict('GLMM Parameters')

glmm_par.push_param(UVNParam('mu', min_info=vp_base['mu_info_min'][0]))
glmm_par.push_param(GammaParam('tau',
                               min_shape=vp_base['tau_alpha_min'][0],
                               min_rate=vp_base['tau_beta_min'][0]))
glmm_par.push_param(MVNParam('beta', K, min_info=vp_base['beta_diag_min'][0]))
glmm_par.push_param(UVNParamVector('u', NG, min_info=vp_base['u_info_min'][0]))

# Initialize with ADVI.  Don't forget to add the ADVI computation time to your final VB time!
advi_fit = json_dat['advi_results']
glmm_par['mu'].mean.set(advi_fit['mu_mean'][0])
glmm_par['mu'].info.set(1 / advi_fit['mu_var'][0])

tau_mean = advi_fit['tau_mean'][0]
tau_var = advi_fit['tau_var'][0]
glmm_par['tau'].shape.set((tau_mean ** 2) / tau_var)
glmm_par['tau'].rate.set(tau_var / tau_mean)

glmm_par['beta'].mean.set(np.array(advi_fit['beta_mean']))
glmm_par['beta'].info.set(np.array(advi_fit['beta_info']))

glmm_par['u'].mean.set(np.array(advi_fit['u_mean']))
glmm_par['u'].info.set(1 / np.array(advi_fit['u_var']))

free_par_vec = glmm_par.get_free()

In [4]:
print glmm_par['tau'].shape._ScalarParam__lb
print glmm_par['u'].info._VectorParam__lb

1e-06
0.001


In [5]:
# Define moment parameters

moment_par = ModelParamsDict('Moment Parameters')
moment_par.push_param(VectorParam('e_beta', K))
moment_par.push_param(PosDefMatrixParam('e_beta_outer', K))
moment_par.push_param(ScalarParam('e_mu'))
moment_par.push_param(ScalarParam('e_mu2'))
moment_par.push_param(ScalarParam('e_tau'))
moment_par.push_param(ScalarParam('e_log_tau'))
moment_par.push_param(VectorParam('e_u', NG))
moment_par.push_param(VectorParam('e_u2', NG))

def set_moments(glmm_par, moment_par):
    moment_par['e_beta'].set(glmm_par['beta'].e())
    moment_par['e_beta_outer'].set(glmm_par['beta'].e_outer())
    moment_par['e_mu'].set(glmm_par['mu'].e())
    moment_par['e_mu2'].set(glmm_par['mu'].e_outer())
    moment_par['e_tau'].set(glmm_par['tau'].e())
    moment_par['e_log_tau'].set(glmm_par['tau'].e_log())
    moment_par['e_u'].set(glmm_par['u'].e())
    moment_par['e_u2'].set((glmm_par['u'].e_outer()))
    
set_moments(glmm_par, moment_par)

# Moment indices.
moment_indices = copy.deepcopy(moment_par)
moment_indices.set_vector(1 + np.array(range(moment_indices.vector_size())))

In [6]:
def decode_combined_parameters(combined_free_par_vec, glmm_par, prior_par):
    assert glmm_par.free_size() + prior_par.vector_size() == len(combined_free_par_vec) 
    glmm_par.set_free(combined_free_par_vec[0:glmm_par.free_size()])
    prior_par.set_vector(combined_free_par_vec[glmm_par.free_size():])

    
def encode_combined_parameters(glmm_par, prior_par):
    combined_free_par_vec = np.full(glmm_par.free_size() + prior_par.vector_size(), float('nan'))
    combined_free_par_vec[0:glmm_par.free_size()] = glmm_par.get_free()
    combined_free_par_vec[glmm_par.free_size():] = prior_par.get_vector()
    return combined_free_par_vec


In [7]:
def inf_like(val):
    return 1 / np.zeros_like(1, val)[0]


def ELogPrior(prior_par, glmm_par_elbo):
    e_beta = glmm_par_elbo['beta'].mean.get()
    info_beta = glmm_par_elbo['beta'].info.get()
    cov_beta = np.linalg.inv(info_beta)
    beta_prior_info = prior_par['beta_prior_info'].get()
    beta_prior_mean = prior_par['beta_prior_mean'].get()
    e_log_p_beta = MVNPrior(beta_prior_mean, beta_prior_info, e_beta, cov_beta)
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    info_mu = glmm_par_elbo['mu'].info.get()
    var_mu = 1 / info_mu
    e_log_p_mu = UVNPrior(prior_par['mu_prior_mean'].get(), prior_par['mu_prior_info'].get(), e_mu, var_mu) 

    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    tau_prior_shape = prior_par['tau_prior_alpha'].get()
    tau_prior_rate = prior_par['tau_prior_beta'].get()
    e_log_p_tau = GammaPrior(tau_prior_shape, tau_prior_rate, e_tau, e_log_tau)
    
    return  e_log_p_beta + e_log_p_mu + e_log_p_tau
           

def DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta, e_u, var_u, std_draws):
    z_mean = e_u + np.matmul(x_mat, e_beta)
    z_sd = np.sqrt(var_u + np.einsum('nk,kj,nj->n', x_mat, cov_beta, x_mat))
    z = np.einsum('i,j->ij', z_sd, std_draws) + np.expand_dims(z_mean, 1)

    # The sum is over observations and draws, so dividing by the draws size
    # gives the sum of sample expectations over the draws.
    # p = exp(z) / (1 + exp(z))
    # log(1 - p) = log(1 / (1 + exp(z))) = -log(1 + exp(z))
    logit_term = -np.sum(np.log1p(np.exp(z))) / std_draws.size
    y_term = np.sum(y_vec * z_mean)
    return y_term + logit_term


def RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau):
    return -0.5 * e_tau * np.sum(((e_mu - e_u) ** 2) + var_mu + var_u) + 0.5 * e_log_tau * len(e_u)

    
def Elbo(y_vec, x_mat, y_g_vec, glmm_par_elbo, std_draws, prior_par):
    e_beta = glmm_par_elbo['beta'].mean.get()
    info_beta = glmm_par_elbo['beta'].info.get()
    cov_beta = np.linalg.inv(info_beta)
    
    e_u = glmm_par_elbo['u'].mean.get()
    info_u = glmm_par_elbo['u'].info.get()
    var_u = 1 / info_u
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    info_mu = glmm_par_elbo['mu'].info.get()
    var_mu = 1 / info_mu
    
    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
        
    ll = \
        DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta,
                          e_u[y_g_vec], var_u[y_g_vec], std_draws) + \
        RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)
    if np.isnan(ll):
        return -np.inf

    e_log_prior = ELogPrior(prior_par, glmm_par_elbo)
    if np.isnan(e_log_prior):
        return -np.inf
    
    tau_shape = glmm_par_elbo['tau'].shape.get()
    tau_rate = glmm_par_elbo['tau'].rate.get()
    entropy = \
        UnivariateNormalEntropy(info_mu) + \
        MultivariateNormalEntropy(info_beta) + \
        UnivariateNormalEntropy(info_u) + \
        GammaEntropy(tau_shape, tau_rate)

    return ll[0] + e_log_prior[0] + entropy


class KLWrapper(object):
    def __init__(self, glmm_par, prior_par, x_mat, y_vec, y_g_vec, num_draws):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.__prior_par_ad = copy.deepcopy(prior_par)
        self.x_mat = x_mat
        self.y_vec = y_vec
        self.y_g_vec = y_g_vec
        draw_spacing = 1 / float(num_draws + 1)
        target_quantiles = np.linspace(draw_spacing, 1 - draw_spacing, num_draws)
        self.std_draws = sp.stats.norm.ppf(target_quantiles)

    def Eval(self, free_par_vec, verbose=False):
        self.__glmm_par_ad.set_free(free_par_vec)
        kl = -Elbo(self.y_vec, self.x_mat, self.y_g_vec,
                   self.__glmm_par_ad, self.std_draws, self.__prior_par_ad)
        if verbose: print kl
            
        # TODO: this is returning an array when it should be a scalar.
        return kl
    
    def ExpectedLogPrior(self, combined_free_par_vec):
        # Encode the glmm parameters first and the prior second.
        decode_combined_parameters(combined_free_par_vec, self.__glmm_par_ad, self.__prior_par_ad)
        e_log_prior = ELogPrior(self.__prior_par_ad, self.__glmm_par_ad)
        return e_log_prior[0]
        

class MomentWrapper(object):
    def __init__(self, glmm_par, moment_par):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.__moment_par = copy.deepcopy(moment_par)

    # Return a posterior moment of interest as a function of unconstrained parameters.
    def GetMoments(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        set_moments(self.__glmm_par_ad, self.__moment_par)
        return self.__moment_par.get_vector()
    
    def GetMomentParameters(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        set_moments(self.__glmm_par_ad, self.__moment_par)
        return self.__moment_par


kl_wrapper = KLWrapper(glmm_par, prior_par, x_mat, y_vec, y_g_vec, 10)
KLGrad = grad(kl_wrapper.Eval)
KLHess = hessian(kl_wrapper.Eval)
KLHessVecProd = hessian_vector_product(kl_wrapper.Eval)  
print kl_wrapper.Eval(free_par_vec)

moment_wrapper = MomentWrapper(glmm_par, moment_par)
MomentJacobian = jacobian(moment_wrapper.GetMoments)

combined_free_par_vec = encode_combined_parameters(glmm_par, prior_par)
PriorHess = hessian(kl_wrapper.ExpectedLogPrior)
kl_wrapper.ExpectedLogPrior(combined_free_par_vec)


609255242.588


-5789972.763644468

In [8]:
import timeit

time_num = 10

print 'Function time:'
print timeit.timeit(lambda: kl_wrapper.Eval(free_par_vec), number=time_num) / time_num

print 'Grad time:'
print timeit.timeit(lambda: KLGrad(free_par_vec), number=time_num) / time_num

print 'Hessian vector product time:'
print timeit.timeit(lambda: KLHessVecProd(free_par_vec, free_par_vec + 1), number=time_num) / time_num

# print 'Moment jacobian time:'
# print timeit.timeit(lambda: MomentJacobian(free_par_vec), number=time_num) / time_num

# time_num = 1
# print 'Prior Hessian time:'
# print timeit.timeit(lambda: PriorHess(combined_free_par_vec), number=time_num) / time_num

# so slow
# print 'Hessian time:'
# print timeit.timeit(lambda: KLHess(free_par_vec), number=time_num) / time_num


Function time:
0.0206583976746
Grad time:
0.0409413814545
Hessian vector product time:
0.0825641870499


In [9]:
import time

class OptimizationPath(object):
    def __init__(self):
        self.x_history = []
        pass
    
    def save(self, x):
        self.x_history.append(x)

bfgs_path = OptimizationPath()
init_par_vec = copy.deepcopy(free_par_vec)

# Optimize.
vb_time = time.time()
# print 'Running BFGS'
# vb_opt_bfgs = optimize.minimize(
#     lambda par: kl_wrapper.Eval(par, verbose=True), init_par_vec,
#     method='bfgs', jac=KLGrad, tol=1e-2, callback=bfgs_path.save,
#     options={'maxiter': 100, 'gtol': 1e-2, 'disp': True})

trust_path = OptimizationPath()
print 'Running Newton Trust Region'
# trust_init = copy.deepcopy(vb_opt_bfgs.x)
trust_init = copy.deepcopy(init_par_vec)
vb_opt = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True),
    trust_init, method='trust-ncg', jac=KLGrad, hessp=KLHessVecProd,
    tol=1e-6, callback=trust_path.save, options={'maxiter': 100, 'disp': True, 'gtol': 1e-6 })

vb_time = time.time() - vb_time

glmm_par_opt = copy.deepcopy(glmm_par)
glmm_par_opt.set_free(vb_opt.x)

print 'Done.'

print vb_time / 60

Running Newton Trust Region
609255242.588
224014466.07
82383044.6089
30300338.5125
11144915.9478
4099215.3155
1507637.69336
554462.411007
203982.242169
75213.2779358
28006.9748586
10807.8479768
4650.26205266
2554.40273042
1938.07732647
1812.43223317
1730.34395129
1681.70941656
1660.32887607
1649.22693388
1644.78379579
1644.31936791
1644.26495942
1640.38679467
1640.33164449
1639.97406248
1639.24330499
1639.24246142
1639.23397031
1639.07527383
1639.06804101
1639.05499256
1639.05381341
1639.05376167
1639.05376148
Optimization terminated successfully.
         Current function value: 1639.053761
         Iterations: 34
         Function evaluations: 35
         Gradient evaluations: 35
         Hessian evaluations: 0
Done.
0.587432952722


In [10]:
print len(trust_path.x_history)

# for i in range(len(path.x_history) - 1):
#     print np.sum(path.x_history[i + 1] - path.x_history[i])

glmm_par.set_free(trust_path.x_history[len(trust_path.x_history) - 1])
# print glmm_par
    
for i in range(len(trust_path.x_history)):
    glmm_par.set_free(trust_path.x_history[i])
    print 'Iteration ' + str(i) + '\n'
    print str(glmm_par['tau']) + '\n'


34
Iteration 0

tau:
tau_shape: [ 793.03053848]
tau_rate: [ 0.00111538]

Iteration 1

tau:
tau_shape: [ 481.18719791]
tau_rate: [ 0.00183676]

Iteration 2

tau:
tau_shape: [ 292.03452586]
tau_rate: [ 0.00302497]

Iteration 3

tau:
tau_shape: [ 177.26119529]
tau_rate: [ 0.00498212]

Iteration 4

tau:
tau_shape: [ 107.6048248]
tau_rate: [ 0.00820574]

Iteration 5

tau:
tau_shape: [ 65.32508746]
tau_rate: [ 0.0135152]

Iteration 6

tau:
tau_shape: [ 39.66110148]
tau_rate: [ 0.02225919]

Iteration 7

tau:
tau_shape: [ 24.08359501]
tau_rate: [ 0.03665527]

Iteration 8

tau:
tau_shape: [ 14.63045394]
tau_rate: [ 0.0603385]

Iteration 9

tau:
tau_shape: [ 8.89778191]
tau_rate: [ 0.09921749]

Iteration 10

tau:
tau_shape: [ 5.42828539]
tau_rate: [ 0.16267264]

Iteration 11

tau:
tau_shape: [ 3.34104773]
tau_rate: [ 0.26460639]

Iteration 12

tau:
tau_shape: [ 2.10881876]
tau_rate: [ 0.42142371]

Iteration 13

tau:
tau_shape: [ 1.42623704]
tau_rate: [ 0.63695399]

Iteration 14

tau:
tau_shape: 

In [11]:
# print(glmm_par_opt)
if simulate_data:
    print true_beta
    print glmm_par_opt['beta']
    print '---------------\n'
    print true_tau
    print glmm_par_opt['tau'].e()

    e_u = glmm_par_opt['u'].e()
    info_u = glmm_par_opt['u'].info.get()
    var_u = 1 / info_u
    e_beta = glmm_par_opt['beta'].e()
    e_beta_outer = glmm_par_opt['beta'].e_outer()
    std_draws = kl_wrapper.std_draws

    rho_mean = e_u[y_g_vec] + np.matmul(x_mat, e_beta)
    rho_sd = np.sqrt(var_u[y_g_vec] + np.einsum('nk,kj,nj->n', x_mat, e_beta_outer, x_mat))
    z = np.einsum('i,j->ij', rho_sd, std_draws) + np.expand_dims(rho_mean, 1)
    logit_term = -np.einsum('ij->i', np.log1p(np.exp(z))) / std_draws.size

    print rho_sd
    print var_u[y_g_vec]
    # print np.mean(var_u)


In [12]:
# Check the random effect estimates.  This requires simulated data.
if simulate_data:
    from ggplot import *
    import pandas as pd
    %matplotlib inline
    
    print glmm_par_opt['mu'].e()
    print true_mu

    print glmm_par_opt['tau'].e()
    print true_tau

    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['u'].mean.get(), 'true': true_u })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['beta'].mean.get(), 'true': true_beta })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': logit_term, 'true': np.log(1 - true_rho) })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)

In [13]:
# LRVB with conjugate gradient.  This turns out to be way slower with any appreciable number of moments.
if False:
    from scipy.sparse.linalg import LinearOperator
    import sys

    # This will actually compute Hess^1 * moment_jac.T, leading to perhaps confusing
    # naming of "columns".  
    ObjHessVecProdLO = LinearOperator((vb_opt.x.size, vb_opt.x.size), lambda par: KLHessVecProd(vb_opt.x, par))
    # print moment_jac.T.shape
    # print ObjHessVecProdLO.shape
    # cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, moment_jac.T)

    cg_time = time.time()
    lrvb_term = np.full(moment_jac.T.shape, float('nan'))
    for col in range(moment_jac.shape[0]):
        sys.stdout.write('.')
        sys.stdout.flush()
        cg_res, info = sp.sparse.linalg.cg(ObjHessVecProdLO, moment_jac[col, :])
        assert info == 0
        lrvb_term[:, col] = cg_res
    cg_time = time.time() - cg_time

    print 'all done dude'
else:
    cg_time = float('inf')

In [14]:
# Slow, but probably faster than using CG.
combined_free_par_vec = encode_combined_parameters(glmm_par_opt, prior_par)

hess_time = time.time()
print 'KL Hessian:\n'
kl_hess = KLHess(vb_opt.x)
print 'Log prior Hessian:\n'
log_prior_hess_full = PriorHess(combined_free_par_vec)
hess_time =  time.time() - hess_time
elbo_hess = -kl_hess

print 'hess_time: %f' % hess_time
print 'cg_time: %f' % cg_time

KL Hessian:

Log prior Hessian:

hess_time: 68.996338
cg_time: inf


In [15]:
glmm_inds = range(glmm_par_opt.free_size())
prior_inds = range(glmm_par_opt.free_size(), len(combined_free_par_vec))
log_prior_hess = log_prior_hess_full[np.ix_(prior_inds, glmm_inds)]

moment_jac = MomentJacobian(vb_opt.x)
lrvb_cov = np.matmul(moment_jac, np.linalg.solve(kl_hess, moment_jac.T))

prior_indices = copy.deepcopy(prior_par)
prior_indices.set_vector(1 + np.array(range(prior_indices.vector_size())))

vp_indices = copy.deepcopy(glmm_par_opt)
vp_indices.set_vector(1 + np.array(range(vp_indices.vector_size())))

In [16]:
if not simulate_data:
    run_name = 'debug'
    result_dict = { 'glmm_par_opt': glmm_par_opt.dictval(), 'run_name': run_name,
                    'vb_time': vb_time,'hess_time': hess_time, 
                    'moment_indices': moment_indices.dictval(),
                    'prior_indices': prior_indices.dictval(),
                    'vp_indices': vp_indices.dictval(),
                    'lrvb_cov': lrvb_cov.tolist(), 'moment_jac': moment_jac.tolist(),
                    'elbo_hess': elbo_hess.tolist(), 'log_prior_hess': log_prior_hess.tolist() }

    result_json = json.dumps(result_dict)
    json_file = open(json_output_filename, 'w')
    json_file.write(result_json)
    json_file.close()

    print(json_output_filename)

/home/rgiordan/Documents/git_repos/LRVBLogitGLMM/LogitGLMMLRVB/inst/data/simulated_data_large_python_vb_results.json


In [32]:
import math

mu_ind = moment_indices['e_mu'].get() - 1 # moment indices is one-indexed
print moment_par['e_mu']
print math.sqrt(lrvb_cov[mu_ind, mu_ind])

e_mu: -0.5253
0.0706139471208
